In [1]:
import gym
import gym_Physics_RL
from torch import nn as nn

from rlkit.exploration_strategies.base import PolicyWrappedWithExplorationStrategy
from rlkit.exploration_strategies.epsilon_greedy import EpsilonGreedy
from rlkit.policies.argmax import ArgmaxDiscretePolicy
from rlkit.torch.dqn.double_dqn import DoubleDQNTrainer
from rlkit.torch.networks import Mlp
import rlkit.torch.pytorch_util as ptu
from rlkit.data_management.env_replay_buffer import EnvReplayBuffer
from rlkit.launchers.launcher_util import setup_logger
from rlkit.samplers.data_collector import MdpPathCollector
from rlkit.torch.torch_rl_algorithm import TorchBatchRLAlgorithm


def experiment(variant):
    expl_env = gym.make('Physics_RL-v0')
    eval_env = gym.make('Physics_RL-v0')
    obs_dim = expl_env.observation_space.low.size
    action_dim = eval_env.action_space.n

    qf = Mlp(
        hidden_sizes=[32, 32],
        input_size=obs_dim,
        output_size=action_dim,
    )
    target_qf = Mlp(
        hidden_sizes=[32, 32],
        input_size=obs_dim,
        output_size=action_dim,
    )
    qf_criterion = nn.MSELoss()
    eval_policy = ArgmaxDiscretePolicy(qf)
    expl_policy = PolicyWrappedWithExplorationStrategy(
        EpsilonGreedy(expl_env.action_space),
        eval_policy,
    )
    eval_path_collector = MdpPathCollector(
        eval_env,
        eval_policy,
    )
    expl_path_collector = MdpPathCollector(
        expl_env,
        expl_policy,
    )
    trainer = DoubleDQNTrainer(
        qf=qf,
        target_qf=target_qf,
        qf_criterion=qf_criterion,
        **variant['trainer_kwargs']
    )
    replay_buffer = EnvReplayBuffer(
        variant['replay_buffer_size'],
        expl_env,
    )
    algorithm = TorchBatchRLAlgorithm(
        trainer=trainer,
        exploration_env=expl_env,
        evaluation_env=eval_env,
        exploration_data_collector=expl_path_collector,
        evaluation_data_collector=eval_path_collector,
        replay_buffer=replay_buffer,
        **variant['algorithm_kwargs']
    )
    algorithm.to(ptu.device)
    algorithm.train()

doodad not detected


In [2]:
if __name__ == "__main__":
    # noinspection PyTypeChecker
    variant = dict(
        algorithm="DDQN",
        version="normal",
        layer_size=256,
        replay_buffer_size=int(1E6),
        algorithm_kwargs=dict(
            num_epochs=3000,
            num_eval_steps_per_epoch=5000,
            num_trains_per_train_loop=1000,
            num_expl_steps_per_train_loop=10,
            min_num_steps_before_training=1000,
            max_path_length=100,
            batch_size=16,
        ),
        trainer_kwargs=dict(
            discount=0.99,
            learning_rate=3E-5,
        ),
    )
    setup_logger('./ddqn-train', variant=variant)
    # ptu.set_gpu_mode(True)  # optionally set the GPU (default=False)
    experiment(variant)

2019-06-09 01:34:02.724874 Pacific Daylight Time | Variant:
2019-06-09 01:34:02.727877 Pacific Daylight Time | {
  "algorithm": "DDQN",
  "version": "normal",
  "layer_size": 256,
  "replay_buffer_size": 1000000,
  "algorithm_kwargs": {
    "num_epochs": 3000,
    "num_eval_steps_per_epoch": 5000,
    "num_trains_per_train_loop": 1000,
    "num_expl_steps_per_train_loop": 10,
    "min_num_steps_before_training": 1000,
    "max_path_length": 100,
    "batch_size": 16
  },
  "trainer_kwargs": {
    "discount": 0.99,
    "learning_rate": 3e-05
  }
}
2019-06-09 01:34:09.217871 Pacific Daylight Time | [ddqn-train_2019_06_09_01_34_02_0000--s-0] Epoch 0 finished
---------------------------------------------  --------------
replay_buffer/size                             1010
trainer/QF Loss                                   1.97087
trainer/Y Predictions Mean                        0.226821
trainer/Y Predictions Std                         0.082281
trainer/Y Predictions Max                     

evaluation/env_infos/X_pos_2 Min                133
time/data storing (s)                             0.000129322
time/evaluation sampling (s)                      3.7404
time/exploration sampling (s)                     0.00771993
time/logging (s)                                  0.040161
time/saving (s)                                   0.00782462
time/training (s)                                 1.79598
time/epoch (s)                                    5.59221
time/total (s)                                    8.28735
Epoch                                             0
---------------------------------------------  --------------
2019-06-09 01:34:15.564566 Pacific Daylight Time | [ddqn-train_2019_06_09_01_34_02_0000--s-0] Epoch 1 finished
---------------------------------------------  -------------
replay_buffer/size                             1020
trainer/QF Loss                                   0.117002
trainer/Y Predictions Mean                       -0.985309
trainer/Y Predicti

evaluation/env_infos/X_pos_2 Max                580
evaluation/env_infos/X_pos_2 Min                123
time/data storing (s)                             0.00015765
time/evaluation sampling (s)                      3.85624
time/exploration sampling (s)                     0.0073492
time/logging (s)                                  0.0465634
time/saving (s)                                   0.00693086
time/training (s)                                 2.25099
time/epoch (s)                                    6.16823
time/total (s)                                   14.6384
Epoch                                             1
---------------------------------------------  -------------
2019-06-09 01:34:22.410624 Pacific Daylight Time | [ddqn-train_2019_06_09_01_34_02_0000--s-0] Epoch 2 finished
---------------------------------------------  ---------------
replay_buffer/size                              1030
trainer/QF Loss                                   37.4872
trainer/Y Predictions Mea

evaluation/env_infos/initial/X_pos_2 Max         580
evaluation/env_infos/initial/X_pos_2 Min         121
evaluation/env_infos/X_pos_2 Mean                401.086
evaluation/env_infos/X_pos_2 Std                 127.393
evaluation/env_infos/X_pos_2 Max                 580
evaluation/env_infos/X_pos_2 Min                 121
time/data storing (s)                              0.000145334
time/evaluation sampling (s)                       4.28354
time/exploration sampling (s)                      0.0084273
time/logging (s)                                   0.0348846
time/saving (s)                                    0.00872289
time/training (s)                                  2.26127
time/epoch (s)                                     6.59699
time/total (s)                                    21.4715
Epoch                                              2
---------------------------------------------  ---------------
2019-06-09 01:34:28.870639 Pacific Daylight Time | [ddqn-train_2019_06_09_01

evaluation/env_infos/final/X_pos_2 Max           578
evaluation/env_infos/final/X_pos_2 Min           121
evaluation/env_infos/initial/X_pos_2 Mean        331.561
evaluation/env_infos/initial/X_pos_2 Std         135.3
evaluation/env_infos/initial/X_pos_2 Max         578
evaluation/env_infos/initial/X_pos_2 Min         121
evaluation/env_infos/X_pos_2 Mean                357.422
evaluation/env_infos/X_pos_2 Std                 132.465
evaluation/env_infos/X_pos_2 Max                 578
evaluation/env_infos/X_pos_2 Min                 121
time/data storing (s)                              0.000144513
time/evaluation sampling (s)                       4.26719
time/exploration sampling (s)                      0.0081703
time/logging (s)                                   0.0350858
time/saving (s)                                    0.00977102
time/training (s)                                  1.95071
time/epoch (s)                                     6.27107
time/total (s)                  

...

evaluation/env_infos/initial/actions Std            0
evaluation/env_infos/initial/actions Max            0
evaluation/env_infos/initial/actions Min            0
evaluation/env_infos/actions Mean                   0
evaluation/env_infos/actions Std                    0
evaluation/env_infos/actions Max                    0
evaluation/env_infos/actions Min                    0
evaluation/env_infos/final/Y_Pos_1 Mean           149
evaluation/env_infos/final/Y_Pos_1 Std              0
evaluation/env_infos/final/Y_Pos_1 Max            149
evaluation/env_infos/final/Y_Pos_1 Min            149
evaluation/env_infos/initial/Y_Pos_1 Mean         149
evaluation/env_infos/initial/Y_Pos_1 Std            0
evaluation/env_infos/initial/Y_Pos_1 Max          149
evaluation/env_infos/initial/Y_Pos_1 Min          149
evaluation/env_infos/Y_Pos_1 Mean                 149
evaluation/env_infos/Y_Pos_1 Std                    0
evaluation/env_infos/Y_Pos_1 Max                  149
evaluation/env_infos/Y_Pos_1

evaluation/Num Paths                              512
evaluation/Average Returns                        -28.7578
evaluation/env_infos/final/actions Mean             0
evaluation/env_infos/final/actions Std              0
evaluation/env_infos/final/actions Max              0
evaluation/env_infos/final/actions Min              0
evaluation/env_infos/initial/actions Mean           0
evaluation/env_infos/initial/actions Std            0
evaluation/env_infos/initial/actions Max            0
evaluation/env_infos/initial/actions Min            0
evaluation/env_infos/actions Mean                   0
evaluation/env_infos/actions Std                    0
evaluation/env_infos/actions Max                    0
evaluation/env_infos/actions Min                    0
evaluation/env_infos/final/Y_Pos_1 Mean           149
evaluation/env_infos/final/Y_Pos_1 Std              0
evaluation/env_infos/final/Y_Pos_1 Max            149
evaluation/env_infos/final/Y_Pos_1 Min            149
evaluation/env_infos/in

evaluation/Returns Std                              4.16037
evaluation/Returns Max                            -26
evaluation/Returns Min                            -61
evaluation/Actions Mean                            48.3947
evaluation/Actions Std                             29.3556
evaluation/Actions Max                             98
evaluation/Actions Min                              1
evaluation/Num Paths                              406
evaluation/Average Returns                        -31.3005
evaluation/env_infos/final/actions Mean             0
evaluation/env_infos/final/actions Std              0
evaluation/env_infos/final/actions Max              0
evaluation/env_infos/final/actions Min              0
evaluation/env_infos/initial/actions Mean           0
evaluation/env_infos/initial/actions Std            0
evaluation/env_infos/initial/actions Max            0
evaluation/env_infos/initial/actions Min            0
evaluation/env_infos/actions Mean                   0
evaluat

evaluation/path length Max                        100
evaluation/path length Min                          9
evaluation/Rewards Mean                            -1.97777
evaluation/Rewards Std                              4.19782
evaluation/Rewards Max                             -1
evaluation/Rewards Min                            -20
evaluation/Returns Mean                           -37.9885
evaluation/Returns Std                             15.7889
evaluation/Returns Max                            -28
evaluation/Returns Min                           -119
evaluation/Actions Mean                            30.8054
evaluation/Actions Std                             24.0668
evaluation/Actions Max                             98
evaluation/Actions Min                              1
evaluation/Num Paths                              260
evaluation/Average Returns                        -37.9885
evaluation/env_infos/final/actions Mean             0
evaluation/env_infos/final/actions Std       

KeyboardInterrupt: 

In [4]:
env = gym.make('Physics_RL-v0')
env.reset()

(10, 174, 421, 10)

In [21]:
#env.observation_space.low.size
env.step(10)

((180, 174, 421, 95),
 0,
 False,
 {'Velocity_y': 5,
  'X_pos_2': 421,
  'Y_Pos_1': 278,
  'actions': [10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10]})